In [1]:
!python -V

Python 3.12.3


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer # turns dictionary into vector
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [27]:
df_jan2023 = pd.read_parquet('/home/mlops-zoomcamp/notebooks/data/yellow_tripdata_2023-01.parquet')
df_feb2023 = pd.read_parquet('/home/mlops-zoomcamp/notebooks/data/yellow_tripdata_2023-02.parquet')

print("January 2023 columns count: ", df_jan2023.columns.size)

January 2023 columns count:  19


Q2. Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [28]:
df_jan2023['tpep_dropoff_datetime'] = pd.to_datetime(df_jan2023['tpep_dropoff_datetime'])
df_jan2023['tpep_pickup_datetime'] = pd.to_datetime(df_jan2023['tpep_pickup_datetime'])

df_jan2023['duration'] = (df_jan2023['tpep_dropoff_datetime'] - df_jan2023['tpep_pickup_datetime'])
df_jan2023["duration"] = df_jan2023["duration"].apply(lambda td: td.total_seconds()/60)
jan2023_std_dev = df_jan2023["duration"].std()

print("January 2023 standard deviation: ", jan2023_std_dev)

January 2023 standard deviation:  42.594351241920904


Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [29]:
tot_trips0 = len(df_jan2023)
# dropping outliers
df_jan2023 = df_jan2023[(df_jan2023["duration"] >= 1) & (df_jan2023["duration"] <= 60)]
tot_trips1 = len(df_jan2023)

print("Fraction remaining after dropping outliers: ", 100 * (tot_trips1/tot_trips0))

Fraction remaining after dropping outliers:  98.1220282212598


Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [30]:
dv = DictVectorizer() #one-hot encoding

l_features = ['PULocationID', 'DOLocationID']
df_jan2023[l_features] = df_jan2023[l_features].astype(str)
dict_features = df_jan2023[l_features].to_dict(orient='records')

X_train = dv.fit_transform(dict_features)
col_number = X_train.shape[1]
print("The dimensionality (number of columns) is:", col_number)

The dimensionality (number of columns) is: 515


Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [31]:
target = 'duration'
y_train = df_jan2023[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse = root_mean_squared_error(y_train, y_pred)

print("RMSE on train: ", rmse)

RMSE on train:  7.649261920265976


Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [ ]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

df_val = read_dataframe('/home/mlops-zoomcamp/notebooks/data/yellow_tripdata_2023-02.parquet')
df_train = df_jan2023.copy()

categorical = ['PULocationID', 'DOLocationID']
train_dicts = df_train[categorical].to_dict(orient='records')
val_dicts = df_val[categorical].to_dict(orient='records')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse_model = root_mean_squared_error(y_val, y_pred)

print("RMSE on validation set: ", rmse_model)